In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jan  3 00:19:43 2022

@author: PythonCat_879
"""

path = "url_list.txt"
f = open(path, "r")
level_0 = f.readlines()
for i in range(len(level_0)):
    level_0[i] = level_0[i][:-1]


from aiohttp import ClientSession
from bs4 import BeautifulSoup
import asyncio
import time
import aiohttp
from fake_useragent import UserAgent # 作假標頭 # 不得不說這一行程式碼真的是偉大的發明


'''
在執行非同步(Asynchronous)的程式碼時，有一點非常重要的就是要建立任務清單(Task)
讓事件迴圈(Event Loop)能夠知道有哪些非同步的任務需要執行，進而在多項任務中，利用等待的空檔切換執行其它的任務，提升效率。
'''
'''
為了讓程式碼易於維護，所以再定義一個fetch協程(coroutine)
用來接收請求的網址及用戶端的session，非同步發送請求(request)與解析所收到的HTML原始碼回應，爬取想要的資料，如下。
'''
'''
為何使用with語法?
with語法可以自動釋放程式空間
'''
'''
Note of Coroutine 
所謂 Coroutine 就是一個可以暫停將執行權讓給其他 Coroutine 或 Awaitables obj 的函數，等其執行完後再繼續執行
並可以多次的進行這樣的暫停與繼續。

async：用來宣告 function 能夠有異步的功能
await：用來標記 Coroutine 切換暫停和繼續的點

注意： await 後面必須接一個 Coroutine 對象或是 awaitable 類型的對象
await 的目的是將控制權回傳給事件循環 (event loop) 並等待返回，而背後實現暫停掛起函數操作的是 yield
使用方法：加在要等待的 function 前面
'''

#定義協程(coroutine)
async def main(lv_name,connector,timeout): # main為非同步程式最高級進入點
    # 給定初始資料
    links = list() #  空串列設置
    for original_url in upper_level:
        links.append(original_url)
    async with ClientSession(connector=connector, timeout=timeout) as session:
        #為何要用session？
        #要執行大量的 HTTP 要求，建議使用 ClientSession ，該類別不僅會建立 connection pool
        #更實作了 keep-alive 等功能，可以有效提升執行效率
        #利用Python Comprehesion語法，來建立任務清單(Task)，接著
        #一個session內部包含一個連接池。連接重用和保持連接（預設情況下均處於啟用狀態）可能會提高總體性能。
        
        #透過asyncio模組(Module)的gather()方法(Method)進行打包及執行
        #也由於呼叫的gather()為非同步的方法(Method)，所以要加上await關鍵字，來進行非同步等待。
        
        tasks = [asyncio.create_task(fetch(link, session)) for link in links]  # 建立任務清單
        results = await asyncio.gather(*tasks)  # 打包任務清單及執行
        print(type(results))
        # 這邊的名字目前沒辦法透過迴圈設定，因為非同步的關係，可會造成無法執行，目前還在找原因
        path = lv_name # 每一層的名字(Asynchronous_crawler_level_XX.txt) 
        f = open(path, 'w' , encoding='UTF-8') # 記得要加編碼器
        print(results,file = f)
        f.close()
        print("花費:" + str(float(time.time() - start_time)) + "秒")
        
        # P.S. 參數加上單引號後，資料被打包成元組
        # 加上await進行非同步等待。
        
#定義協程(coroutine)
async def fetch(link, session):
    # 拿取上面定義的session(也就是任務清單)as response
    aa = []
    try:
        async with session.get(link, headers = headers, ssl=False) as response:  #非同步發送請求 # 加入headers作假標頭
            # 不檢查SSL (不驗證Https，以節省時間)
            html_body = await response.text(encoding="utf-8") # 非同步等待
            
                                              # 這邊因為要解析文檔，所以使用text方法，如果要讀取影像，可以使用read方法
                                              # 1110102 加入utf-8編碼
                                              # 這邊我的理解為，凡是用非同步請求為開頭來接收的函式，其結果都返回後同樣也要非同步接收
                                              # 也就是 "await" 方法。
            soup = BeautifulSoup(html_body, 'html5lib')  # 解析HTML原始碼  # 指定'html5lib'作為解析器
            # 下面就可以開始解析soup物件。
            #若只使用sp.link 則只會找出第一筆
            #故這邊要使用"find.all"語法
            link_search = soup.find_all("a") #找出所有的 <a> (超連結)
            for i in range(len(link_search)):
                k = str(link_search[i].get("href"))  #這行一定要記得要轉type！！！轉成str不然有時候會無法切片取值。
                if k[0:5] == "https": #只留下"https"
                    aa.append(k)
            for url in aa[:10]: # 印出前10 # 測試有沒有get到東西
                print(url) # 印出抓取到的網址  
            
    except Exception as e:
        print(e)
    return aa
def str_to_url(aaa): # aaa 為文字格式。
    bbb = aaa.split(",") # 利用逗號做切分 # bbb為list格式 # 會回傳一個list
    flag_head = False # 一開始起頭開關為False
    tmp_word_list = [] # 裝暫存要轉變成文字的網址，每次只會存一個網址，之後就會被清空
    tmp_url_list = [] # 裝整理好的網址
    for url in bbb: # 在這bbb串列中做遞迴
        for char in url:
            if char == "'": # 當遇見"單引號"時switch (此時Flag_head只有兩種狀態)
                flag_head = not(flag_head) # 轉換開關
                # tmp_word_list.append(char) # 將其收錄至暫存的字串列。
            if flag_head == True:
                tmp_word_list.append(char) # 因為只有h會將其打開，所以一路收錄char 至 tmp_word_list
            if flag_head == False:
                StrA = "".join(tmp_word_list)
                StrA = StrA[1:]  # 去掉前端的單引號。 
                if len(tmp_word_list) != 0: # 不為空才加入
                    tmp_url_list.append(StrA)
                tmp_word_list = [] # 清空tmp_word_list
    # 所以我只要再把這些網址NODE化之後再加入CHILDREN串列就好 =)
    return tmp_url_list #回傳整理好的網址



# 初始化

In [ ]:
# 初始化請用這格。

upper_level = []
now_level = []
new_level = []

path = "Asynchronous_crawler_level_1.txt" # 要導入的檔案名稱！！！
counter =  1 # 他是level幾counter就打幾。
f = open(path, "r",encoding ="utf-8")
tmp_txt = f.readlines() # 這邊整行資料只算一行。
tmp_txt = str(tmp_txt) # 先轉成文字檔，以利處理
tmp_txt = str_to_url(tmp_txt)

for i in tmp_txt:
    upper_level.append(i)
tmp_txt

In [3]:
'''
如果是最一開始的url_list，用這個(全部複製貼上)。
'''
'''
upper_level = []
now_level = []
new_level = []
path = "url_list.txt"
f = open(path, "r",encoding ="utf-8")
level_0 = f.readlines()
for i in level_0:
    upper_level.append(i[:-1])
print(len(level_0))
counter=0
'''





'''
# 格式
# 若不幸從中途開始那請要記得改參數！！！


upper_level = []
now_level = []
new_level = []

path = "Asynchronous_crawler_level_1.txt" # 要導入的檔案名稱！！！
counter =  1 # 他是level幾counter就打幾。
f = open(path, "r",encoding ="utf-8")
tmp_txt = f.readlines() # 這邊整行資料只算一行。
tmp_txt = str(tmp_txt) # 先轉成文字檔，以利處理
tmp_txt = str_to_url(tmp_txt)

for i in tmp_txt:
    upper_level.append(i)
tmp_txt
'''

'\n# 格式\n# 若不幸從中途開始那請要記得改參數！！！\n\n\nupper_level = []\nnow_level = []\nnew_level = []\n\npath = "Asynchronous_crawler_level_1.txt" # 要導入的檔案名稱！！！\ncounter =  1 # 他是level幾counter就打幾。\nf = open(path, "r",encoding ="utf-8")\ntmp_txt = f.readlines() # 這邊整行資料只算一行。\ntmp_txt = str(tmp_txt) # 先轉成文字檔，以利處理\ntmp_txt = str_to_url(tmp_txt)\n\nfor i in tmp_txt:\n    upper_level.append(i)\ntmp_txt\n'

In [4]:
# 目前就一直手動執行下面這三個cell就好，直到演算法停止

In [ ]:
# Step1
# 前面這邊每一層要手動修改
tmp_level = upper_level 
level_name = 'Asynchronous_crawler_level_'+ str((counter+1)) +'.txt' # 這一層要新創的檔案的名字
# 作假標頭
ua = UserAgent()
headers = {'User-Agent': ua.random}
connector = aiohttp.TCPConnector(limit = 30, ssl=False) # 預設最大連線數量 #1110102 跟改限制為0 (也就是不做限制)
timeout =  aiohttp.ClientTimeout(total = 30000*(counter+1), connect=None, # total時間要隨網站數量不同而變動
                      sock_connect=3 , sock_read=3) # 1110102發現，原本預設情況下
                                              # aiohttp 使用總共 300 秒（5 分鐘）的超時時間，這意味著整個操作應該在 5 分鐘內完成。
                                              # 與我發現真實情況相符
                                              # 故我猜測此即為有些request無法完成就中斷的原因
                                              # 1110102將timeout 參數設為 32400 秒，9小時，以令程式盡量全部收到回應。
                                              # connect=300，300秒沒過就time out。
# 整理解釋 aiohttp.ClientTimeout 裡面的參數
# 1. total: 總協程執行時間。
# 2. connect ： 如果超過池(pool)連接限制，則連接建立新連接或等待來自池的空閒連接的最大秒數。
#    (問題：如果pool一直都是被占滿，這樣新建連接要刪除pool中的哪個網址？)
#    (目前解決方法：設定sock_connect的時間比connect的時間短，這樣會先結束長時間無回應的網站，pool就不會誤刪。
# 3. sock_connect：The maximal number of seconds for connecting to a peer for a new connection, not given from a pool.
    #(這邊我的理解為給一個新的連接"最多的"時間限制。)
# 4. sock_read： 讀取網頁資料的最大時間限制(我們這邊都只需要讀網址)，故時間讀取時間應該都很小，故保持預設時間即可
start_time = time.time()  #開始執行時間
loop = asyncio.get_event_loop()  #建立事件迴圈(Event Loop)
loop.create_task(main(level_name,connector,timeout))  #執行協程(coroutine)
print("花費:" + str(float(time.time() - start_time)) + "秒")

# Note: 甚麼時候可以結束？
# 當有輸出txt檔案時代表此cell已經跑完了，可以進行下一步驟。

In [12]:
# Step2

path = 'Asynchronous_crawler_level_' + str((counter+1)) + '.txt' # 這一層找出來的網站
f = open(path, "r", encoding='UTF-8') #記得要加編碼器
now_level = f.readlines() # 這邊整行資料只算一行。
now_level = str(now_level) # 先轉成文字檔，以利處理
now_level = str_to_url(now_level) # 要再轉回為list 格式 以利下面list相加。

# 將重複去掉，因為現在沒有後續節點問題，所以直接用set方法，就可以留下單獨不重複的網站
# 要注意一點，要將上一層的網站與這一層找到的網站相加，再去做set方法去除重複(為防止level_0 與 level_1 中有重複的網站。)
# 加起來在去除重複之後，再將其輸出為一新檔案，進入下一層的迴圈
# 迴圈何時停止？現在先暫時設定，當每一層新增加不重複的網站不超過總數的 10 %，就停止搜尋演算法。 
new_level = upper_level + now_level  # 所以new_level是目前所有的總和
new_level = set(new_level)
new_level = list(new_level) # 再將set 資料 轉成list 資料。
print(f"此層不重複網站總數：{len(new_level)}") # 這邊是不重複網站的總數
print(f"此層增加網站(包含有重複！！！)：{(len(now_level))}") # 此層增加網站總數 
print(f"此層不重複網站增加總數：{(len(set(now_level)))}")
print(f"不重複網站增加佔總數百分比為：{(len(set(now_level))/len(new_level) * 100)}%")

# 創建一個level_note，紀錄每一層的變化。
path = 'level' + str((counter+1)) + 'Note.txt'
fn = open(path, 'w' , encoding='UTF-8')
print("此層不重複網站總數：",len(new_level),file = fn) # 這邊是不重複網站的總數
print("此層增加網站(包含有重複！！！)：",len(now_level),file = fn) # 此層增加網站總數 
print("此層不重複網站增加總數：",(len(set(now_level))),file = fn)
print("不重複網站增加佔總數百分比為：",(len(set(now_level))/len(new_level) * 100),"%",file = fn)
print("參數調整：", file = fn)
print(timeout, file = fn)
print(connector, file = fn)

if (len(set(now_level))/len(new_level) * 100) < 10:
    print("在第",str(counter),"層停止，恭喜發財")
# 沒有跳出迴圈就更新初始值

# 確認有更新 levelXXNote.txt 裡面的文字，再進行下一步。(有時候非同步會比較慢)

此層不重複網站總數：22288
此層增加網站(包含有重複！！！)：121968
此層不重複網站增加總數：10472
不重複網站增加佔總數百分比為：46.984924623115575%


In [8]:
# Step3 更新參數 

upper_level = now_level 
now_level = new_level
new_level = []
counter += 1